In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
from PIL import Image
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras.layers import Activation, Dropout, Flatten, Dense,Conv2D ,GlobalMaxPooling2D, GlobalAveragePooling2D,Conv2D, MaxPooling2D
from keras import backend as K
import cv2
from tensorflow import keras


import kerastuner as kt

In [2]:
path = "../input/photos/" #crear path a los directorios
training_data_dir = path + "train1" 
validation_data_dir = path + "validation1" 
test_data_dir = path + "test1" 

In [3]:
training_data_generator = ImageDataGenerator(
    rescale=1./255,
    zoom_range=1,
    horizontal_flip=True,
    vertical_flip=True)

validation_data_generator = ImageDataGenerator(
    rescale=1./255,
    zoom_range=1,
    horizontal_flip=True,
    vertical_flip=True)

test_data_generator = ImageDataGenerator(
    rescale=1./255,
    zoom_range=1,
    horizontal_flip=True,
    vertical_flip=True)

In [4]:
training_generator = training_data_generator.flow_from_directory(
    training_data_dir,  
    target_size=(500, 500),
    batch_size=9,
    class_mode="categorical")
validation_generator = validation_data_generator.flow_from_directory(
    validation_data_dir,
    target_size=(500, 500),
    batch_size=9,
    class_mode="categorical",
    shuffle= False)  
test_generator = test_data_generator.flow_from_directory(
    test_data_dir,
    target_size=(500, 500),
    batch_size=1,
    class_mode="categorical") 

Found 8245 images belonging to 10 classes.
Found 703 images belonging to 10 classes.
Found 2852 images belonging to 10 classes.


In [7]:
INPUT_SHAPE = (500, 500, 3)
NUM_CLASSES = 10

To put the whole hyperparameter search space together and perform hyperparameter tuning, Keras Tuners uses `HyperModel` instances. Hypermodels are reusable class object introduced with the library, defined as follows:

In [8]:
from kerastuner import HyperModel


class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.Sequential()
        
        model.add(
            Conv2D(
                filters=hp.Choice( #optimize number of filters
                    'num_filters',
                    values=[16, 32, 64]
                ),
                activation='relu',
                kernel_size=3, 
                input_shape=self.input_shape
            )
        )
        model.add(MaxPooling2D((2,2)))
      
        model.add(Flatten())
        
        model.add(
            Dense(
                units=hp.Int( #number of neurons in layer
                    'units',
                    min_value=32,
                    max_value=512,
                    step=32
                ),
                activation=hp.Choice( #different optimization functions
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid']
                )
            )
        )
        model.add(
            Dropout(
                rate=hp.Float( #different drop out
                    'dropout_1',
                    min_value=0.2,
                    max_value=0.7,
                    step=0.1
                )
            )
        )
        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Float(
                    'learning_rate', #different learning rates 
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

hypermodel = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

Next, we need to choose the tuner, there are 3 options: Random Search, Hyperband and Bayesian. Hyperband is an optimized version of random search which uses early-stopping to speed up the hyperparameter tuning process. The main idea is to fit a large number of models for a small number of epochs and to only continue training for the models achieving the highest accuracy on the validation set. The max_epochs variable is the max number of epochs that a model can be trained for.

In [9]:
from kerastuner.tuners import Hyperband

In [10]:
es = tf.keras.callbacks.EarlyStopping( 
    monitor="val_acc",
    patience=5) 

In [ ]:
checkpoint_filepath = "../input/photos"
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_loss",
    verbose=0,
    save_best_only=False,
    save_weights_only=False,
    mode="auto",
    save_freq=3
)

In [11]:
HYPERBAND_MAX_EPOCHS = 20  # The maximum number of epochs to train one model. 
#It is recommended to set this to a value slightly higher than the expected time to convergence for your largest Model,
#and to use early stopping during training

MAX_TRIALS = 10 #Total number of trials (model configurations) to test at most

EXECUTION_PER_TRIAL = 2 #the number of models that should be built and fit for each trial for robustness purposes

tuner = Hyperband(
    hypermodel,
    max_epochs=HYPERBAND_MAX_EPOCHS,
    objective='val_accuracy',
    executions_per_trial=EXECUTION_PER_TRIAL,
    seed=123
)

In [ ]:
tuner.search_space_summary()

In [ ]:
%%time  
tuner.search(training_generator, epochs=20, validation_data = test_generator, callbacks = [es,checkpoint], verbose=1)

In [ ]:
save_model(tuner, 1, 'tuner1.h5')

In [ ]:
tuner.results_summary()

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
un = best_hps.get('units')
un

In [ ]:
best_hps.get('dense_activation')

In [ ]:
best_hps.get('dropout_1')

In [ ]:
best_hps.get('learning_rate')

# tuner was taking too long with this parameters (more than 10 hours only to make 3 trials) so I willsplit it in parts

In [12]:
INPUT_SHAPE = (500, 500, 3)
NUM_CLASSES = 10

## only filters and learning rate

In [ ]:
from kerastuner import HyperModel


class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.Sequential()
        
        model.add(
            Conv2D(
                filters=hp.Choice(
                    'num_filters',
                    values=[16, 32, 64]
                ),
                activation='relu',
                kernel_size=3, 
                input_shape=self.input_shape
            )
        )
        model.add(MaxPooling2D((2,2)))
      
        model.add(Flatten())
        
        model.add(tf.keras.layers.Dense(64, activation='relu'))
               

        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

hypermodel = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

In [ ]:
from kerastuner.tuners import Hyperband

In [ ]:
checkpoint_filepath = "../output/tuner.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_loss",
    verbose=0,
    save_best_only=False,
    save_weights_only=False,
    mode="auto",
    save_freq=3
)

In [ ]:
HYPERBAND_MAX_EPOCHS = 20
MAX_TRIALS = 10
EXECUTION_PER_TRIAL = 2

tuner = Hyperband(
    hypermodel,
    max_epochs=HYPERBAND_MAX_EPOCHS,
    objective='val_accuracy',
    executions_per_trial=EXECUTION_PER_TRIAL,
    seed=123
)

In [ ]:
tuner.search_space_summary()

In [ ]:
%%time
tuner.search(training_generator, epochs=20, validation_data = test_generator, verbose=1) 


### stopped after 8 hours and when saving didn't save the outputs in kaggle
### but I checked the best parameters before , they were: filters 16, learning rate 0.0013969

In [ ]:
tuner.results_summary()

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.save('bestmodel.h5')

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
f = best_hps.get('num_filters')
f

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
lr = best_hps.get('learning_rate')
lr

# choose filters 16, learning rate 0.0013969 best parameters and make a search for dense layer and activation function

In [13]:
INPUT_SHAPE = (500, 500, 3)
NUM_CLASSES = 10

In [15]:
from kerastuner import HyperModel


class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.Sequential()
        
        model.add(
            Conv2D(
                filters=16,
                activation='relu',
                kernel_size=3, 
                input_shape=self.input_shape
            )
        )
        model.add(MaxPooling2D((2,2)))
      
        model.add(Flatten())
        
        model.add(
            Dense(
                units=hp.Int(
                    'units',
                    min_value=32,
                    max_value=128,
                    step=32
                ),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid']
                )
            )
        )
    

        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(
            optimizer=keras.optimizers.Adam(lr = 0.0013969),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

hypermodel = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

In [16]:
from kerastuner.tuners import Hyperband

In [17]:
HYPERBAND_MAX_EPOCHS = 20
MAX_TRIALS = 10
EXECUTION_PER_TRIAL = 2

tuner = Hyperband(
    hypermodel,
    max_epochs=HYPERBAND_MAX_EPOCHS,
    objective='val_accuracy',
    executions_per_trial=EXECUTION_PER_TRIAL,
    seed=123
)

In [18]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': None}
dense_activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}


In [ ]:
%%time
tuner.search(training_generator, epochs=20, validation_data = test_generator, verbose=1)

Trial 9 Complete [00h 57m 23s]
val_accuracy: 0.12272089719772339

Best val_accuracy So Far: 0.3557152897119522
Total elapsed time: 08h 47m 13s

Search: Running Trial #10

Hyperparameter    |Value             |Best Value So Far 
units             |96                |128               
dense_activation  |relu              |relu              
tuner/epochs      |3                 |3                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |2                 |2                 
tuner/round       |0                 |0                 

Epoch 1/3
787/917 [========================>.....] - ETA: 1:01 - loss: 28.0939 - accuracy: 0.2423

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.save('bestmodel.h5')

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
u = best_hps.get('units')
u

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
a = best_hps.get('dense_activation')
a

# choose filters 16, learning rate 0.0013969, dense 128, relu for the first layer. optimize the second dense layer 

In [5]:
INPUT_SHAPE = (500, 500, 3)
NUM_CLASSES = 10

In [6]:
from kerastuner import HyperModel


class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.Sequential()
        
        model.add(
            Conv2D(
                filters=16,
                activation='relu',
                kernel_size=3, 
                input_shape=self.input_shape
            )
        )
        model.add(MaxPooling2D((2,2)))
      
        model.add(Flatten())
        
        model.add(tf.keras.layers.Dense(128, activation='relu'))
        
        model.add(
            Dense(
                units=hp.Int(
                    'units',
                    min_value=128,
                    max_value=256,
                    step=32
                ),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid']
                )
            )
        )

        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(
            optimizer=keras.optimizers.Adam(lr = 0.0013969),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

hypermodel = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

In [7]:
from kerastuner.tuners import Hyperband

In [8]:
HYPERBAND_MAX_EPOCHS = 10
MAX_TRIALS = 10
EXECUTION_PER_TRIAL = 2

tuner = Hyperband(
    hypermodel,
    max_epochs=HYPERBAND_MAX_EPOCHS,
    objective='val_accuracy',
    executions_per_trial=EXECUTION_PER_TRIAL,
    seed=123
)

In [9]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 256, 'step': 32, 'sampling': None}
dense_activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}


In [10]:
%%time
tuner.search(training_generator, epochs=10, validation_data = test_generator, verbose=1)

Trial 12 Complete [00h 40m 22s]
val_accuracy: 0.37447406351566315

Best val_accuracy So Far: 0.41234220564365387
Total elapsed time: 08h 04m 03s

Search: Running Trial #13

Hyperparameter    |Value             |Best Value So Far 
units             |224               |224               
dense_activation  |relu              |relu              
tuner/epochs      |4                 |2                 
tuner/initial_e...|2                 |0                 
tuner/bracket     |2                 |2                 
tuner/round       |1                 |0                 
tuner/trial_id    |faeb44f2b2fcb1e...|None              

Epoch 3/4
917/917 [==============================] - 597s 651ms/step - loss: 8.1254 - accuracy: 0.2270 - val_loss: 1.8567 - val_accuracy: 0.3370
Epoch 4/4
917/917 [==============================] - 599s 653ms/step - loss: 1.8722 - accuracy: 0.3597 - val_loss: 1.8058 - val_accuracy: 0.3580


ResourceExhaustedError: untitled_project/trial_43400bc33ad935806b0b40ab33d11bdb/checkpoints/epoch_0/checkpoint_temp/part-00000-of-00001.data-00000-of-00001.tempstate7204936597837610682; No space left on device [Op:SaveV2]

# choose filters 16, learning rate 0.0013969, dense 128, relu .dense 224 relu . optimize second conv filter 

In [5]:
INPUT_SHAPE = (500, 500, 3)
NUM_CLASSES = 10

In [7]:
from kerastuner import HyperModel


class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.Sequential()
        
        model.add(
            Conv2D(
                filters=16,
                activation='relu',
                kernel_size=3, 
                input_shape=self.input_shape
            )
        )
        model.add(MaxPooling2D((2,2)))
        
        model.add(
            Conv2D(
                filters=hp.Choice(
                    'num_filters',
                    values=[16, 32, 64]
                ),
                activation='relu',
                kernel_size=3, 
                input_shape=self.input_shape
            )
        )
        
        model.add(MaxPooling2D((2,2)))
      
        model.add(Flatten())
        
        model.add(tf.keras.layers.Dense(128, activation='relu'))
        
        model.add(tf.keras.layers.Dense(224, activation='relu'))

        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(
            optimizer=keras.optimizers.Adam(lr = 0.0013969),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

hypermodel = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

In [8]:
from kerastuner.tuners import Hyperband

In [9]:
HYPERBAND_MAX_EPOCHS = 10
MAX_TRIALS = 10
EXECUTION_PER_TRIAL = 2

tuner = Hyperband(
    hypermodel,
    max_epochs=HYPERBAND_MAX_EPOCHS,
    objective='val_accuracy',
    executions_per_trial=EXECUTION_PER_TRIAL,
    seed=123
)

In [10]:
tuner.search_space_summary()

Search space summary
Default search space size: 1
num_filters (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32, 64], 'ordered': True}


In [11]:
%%time
tuner.search(training_generator, epochs=10, validation_data = test_generator, verbose=1)

Trial 3 Complete [00h 45m 07s]
val_accuracy: 0.4374123364686966

Best val_accuracy So Far: 0.4374123364686966
Total elapsed time: 02h 15m 10s
CPU times: user 2h 13min 37s, sys: 2min 35s, total: 2h 16min 12s
Wall time: 2h 15min 10s


# Example of optimized number of filters in the convolutional layer

In [13]:
best_model = tuner.get_best_models(num_models=1)[0]

In [14]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
f = best_hps.get('num_filters')
f

64

Final Model:
    
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=16,kernel_size=3, padding='same', activation='relu', input_shape=(500,500,3)))
model.add(MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(128, activation='relu')) 
model.add(tf.keras.layers.Dense(224, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
            optimizer=tf.keras.optimizers.Adam(lr = 0.0013969),
            metrics=['acc'])